In [55]:
import numpy as np
from PIL import Image
import time, re, os, sys
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.distance import pdist,squareform
from array2gif import write_gif

#from util import *


In [56]:
GIF_path = './GIF'
colormap= np.random.choice(range(256),size=(100,3))


In [57]:
def openImage(path):
	image = cv2.imread(path)
	H, W, C = image.shape
	image_flat = np.zeros((W * H, C))
	for h in range(H):
		image_flat[h * W:(h + 1) * W] = image[h]

	return image_flat,H,W

In [58]:
def precomputed_kernel(X, gamma_s, gamma_c):
    n=len(X)
    # S(x) spacial information
    S=np.zeros((n,2))
    for i in range(n):
        S[i]=[i//100,i%100]
    print(pdist(S,'sqeuclidean').shape)
    K=squareform(np.exp(-gamma_s*pdist(S,'sqeuclidean')))*squareform(np.exp(-gamma_c*pdist(X,'sqeuclidean')))
    print(K.shape)
    return K


In [59]:
def visualize(C,k,H,W):
    '''
    @param C: (10000) belonging classes ndarray
    @param k: #clusters
    @param H: image_H
    @param W: image_W
    @return : (H,W,3) ndarray
    '''
    colors= colormap[:k,:]
    res=np.zeros((H,W,3))
    for h in range(H):
        for w in range(W):
            res[h,w,:] = colors[C[h*W+w]]

    return res.astype(np.uint8)

In [60]:
def kmeans(CLUSTER_NUM, Gram, H, W, init='None'):
    Cluster = np.zeros((CLUSTER_NUM, Gram.shape[1]))
    if init == 'random':
        random_pick=np.random.randint(low=0,high=Gram.shape[0],size=CLUSTER_NUM)
        Cluster = Gram[random_pick,:]
    if init == 'gaussian':
        X_mean=np.mean(Gram,axis=0)
        X_std=np.std(Gram,axis=0)
        for c in range(Gram.shape[1]):
            Cluster[:,c]=np.random.normal(X_mean[c],X_std[c],size=CLUSTER_NUM)
      
    # kmeans++ init
    
    Cluster[0]=Gram[np.random.randint(low=0,high=Gram.shape[0],size=1),:]
    for c in range(1,CLUSTER_NUM):
            Dist=np.zeros((len(Gram),c))
            for i in range(len(Gram)):
                for j in range(c):
                    Dist[i,j]=np.sqrt(np.sum((Gram[i]-Cluster[j])**2))
            Dist_min=np.min(Dist,axis=1)
            sum=np.sum(Dist_min)*np.random.rand()
            for i in range(len(Gram)):
                sum-=Dist_min[i]
                if sum<=0:
                    Cluster[c]=Gram[i]
                    break
    # kmeans++
    diff = 1e9
    eps = 1e-9
    count = 1
    # Classes of each Xi
    C=np.zeros(len(Gram),dtype=np.uint8)
    segments=[]
    while diff > eps:
        # E-step
        for i in range(len(Gram)):
            dist=[]
            for j in range(CLUSTER_NUM):
                dist.append(np.sqrt(np.sum((Gram[i]-Cluster[j])**2)))
            C[i]=np.argmin(dist)
        
        #M-step
        New_Mean=np.zeros(Cluster.shape)
        for i in range(CLUSTER_NUM):
            belong=np.argwhere(C==i).reshape(-1)
            for j in belong:
                New_Mean[i]=New_Mean[i]+Gram[j]
            if len(belong)>0:
                New_Mean[i]=New_Mean[i]/len(belong)

        diff = np.sum((New_Mean - Cluster)**2)
        Cluster=New_Mean
        # visualize
        segment = visualize(C, CLUSTER_NUM, H, W)
        segments.append(segment)
        print('iteration {}'.format(count))
        for i in range(CLUSTER_NUM):
            print('k={}: {}'.format(i + 1, np.count_nonzero(C == i)))
        print('diff {}'.format(diff))
        print('-------------------')
        cv2.imshow('', segment)
        cv2.waitKey(1)
    return C, segments


In [61]:
def plot_eigenvector(xs,ys,zs,C):
    '''
    only for 3-dim datas
    @param xs: (#datapoint) ndarray
    @param ys: (#datapoint) ndarray
    @param zs: (#datapoint) ndarray
    @param C: (#datapoint) ndarray, belonging class
    '''
    fig=plt.figure()
    ax=fig.add_subplot(111,projection='3d')
    markers=['o','^','s']
    for marker,i in zip(markers,np.arange(3)):
        ax.scatter(xs[C==i],ys[C==i],zs[C==i],marker=marker)

    ax.set_xlabel('eigenvector 1st dim')
    ax.set_ylabel('eigenvector 2nd dim')
    ax.set_zlabel('eigenvector 3rd dim')
    plt.show()

In [62]:
def spectral_clustering(path, cut):
    image_flat,H,Width=openImage(path)
    gamma_s=0.001
    gamma_c=0.001
    k_means_initType='k_means_plusplus'
    k=3  # k clusters
    filename = path.split('.')[0] + '_' + cut + '.gif'
    gif_path = os.path.join(GIF_path, filename)
    W=precomputed_kernel(image_flat,gamma_s,gamma_c)
    # degree matrix
    D=np.diag(np.sum(W,axis=1))
    L=D-W
    if cut == 'ratio':
        eigenvalue=np.load('{}_eigenvalue_{:.3f}_{:.3f}_unnormalized.npy'.format(path.split('.')[0],gamma_s,gamma_c))
        eigenvector=np.load('{}_eigenvector_{:.3f}_{:.3f}_unnormalized.npy'.format(path.split('.')[0],gamma_s,gamma_c))
    if cut == 'normalized':
        D_inverse_square_root=np.diag(1/np.diag(np.sqrt(D)))
        L_sym=D_inverse_square_root@L@D_inverse_square_root
        eigenvalue=np.load('{}_eigenvalue_{:.3f}_{:.3f}_normalized.npy'.format(path.split('.')[0],gamma_s,gamma_c))
        eigenvector=np.load('{}_eigenvector_{:.3f}_{:.3f}_normalized.npy'.format(path.split('.')[0],gamma_s,gamma_c))
    sort_index = np.argsort(eigenvalue)
    U = eigenvector[:, sort_index[1:1+k]]
    if cut == 'normalized':
        sums=np.sqrt(np.sum(np.square(U),axis=1)).reshape(-1,1)
        U_norm=U/sums
        C, segments = kmeans(k, U_norm, H, Width, init='gaussian')
    else:
        C, segments = kmeans(k, U, H, Width, init='gaussian')
    # save_gif
    for i in range(len(segments)):
        segments[i] = segments[i].transpose(1, 0, 2)
    write_gif(segments, gif_path, fps=2)
    if k==3:
        plot_eigenvector(U[:,0],U[:,1],U[:,2],C)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print('done')


In [63]:
#spectral_clustering('image1.png', 'ratio')
spectral_clustering('image1.png', 'normalized')
#spectral_clustering('image2.png', 'ratio')
spectral_clustering('image2.png', 'normalized')


(49995000,)
(10000, 10000)
iteration 1
k=1: 3348
k=2: 5407
k=3: 1070
k=4: 175
diff 0.0010939298188603623
-------------------
iteration 1
k=1: 3389
k=2: 5502
k=3: 1056
k=4: 53
diff 0.008098303900681945
-------------------
iteration 1
k=1: 3389
k=2: 5503
k=3: 1066
k=4: 42
diff 0.000458638554963726
-------------------
iteration 1
k=1: 3389
k=2: 5502
k=3: 1069
k=4: 40
diff 1.635242523352633e-05
-------------------
iteration 1
k=1: 3389
k=2: 5502
k=3: 1069
k=4: 40
diff 0.0
-------------------
done
(49995000,)
(10000, 10000)
iteration 1
k=1: 4208
k=2: 1367
k=3: 3299
k=4: 1126
diff 0.22674927559718805
-------------------
iteration 1
k=1: 4122
k=2: 1403
k=3: 3349
k=4: 1126
diff 0.004637213744908798
-------------------
iteration 1
k=1: 4089
k=2: 1430
k=3: 3355
k=4: 1126
diff 0.0003843911287881092
-------------------
iteration 1
k=1: 4083
k=2: 1432
k=3: 3359
k=4: 1126
diff 2.3213789303380792e-05
-------------------
iteration 1
k=1: 4078
k=2: 1437
k=3: 3359
k=4: 1126
diff 7.479696642714751e-06
--